In [1]:
from numpy.random import seed
seed(7532)
from tensorflow import set_random_seed
set_random_seed(7532)

import numpy as np
import pandas as pd

from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata.csv')
valid_set = pd.read_csv('valid_set_metadata.csv')
test_set = pd.read_csv('test_set_metadata.csv')

train_set_partition = np.load('train_set_partition.npy')

Compute the number of parts the train set was partitioned into.

In [3]:
n_parts = len(train_set_partition) - 1
n_parts

20

## CNN Model Training from Scratch with Data Augmentation ##

In [4]:
BEST_MODEL_PATH = 'best_model.hdf5'
INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.0001
N_EPOCHS = 100
BATCH_SIZE = 32

In [5]:
def create_sequential_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', 
                     input_shape=input_shape))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())  
    
    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
 
    model.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(1024, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(5))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [6]:
model = create_sequential_model(INPUT_SHAPE)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 320, 320, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 160, 160, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 160, 160, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 80, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 80, 80, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 80, 80, 128)       73856     
__________

Since our data is divided into 20 parts the network training is performed one part at a time.

Unfortunately, a technical error is causing the training to be stopped after each and every part. Howerver, the trained model does not seem to be affected.

In [9]:
gender_column_position = train_set.columns.get_loc('gender')

X_valid = np.load('valid_set_hmgd_arr.npy')
y_valid = valid_set['gender'].values

batch_limit = train_set_partition[1:] - train_set_partition[:-1]

for part in range(n_parts):        
    if part < 19:
        continue
        
    print(f'Part {part + 1}:')
    
    train_filename = 'train_set_hmgd_arr_' + str(part + 1).zfill(2) + '.npy'
    subrange = range(train_set_partition[part], train_set_partition[part + 1])    
    X_train = np.load(train_filename)
    y_train = train_set.iloc[subrange, gender_column_position].values
    
    # Model initialization/loading
    if part:
        model = load_model(BEST_MODEL_PATH)
    else:
        model = create_sequential_model(X_train.shape[1:])
        adam = Adam(lr=LEARNING_RATE)
        model.compile(optimizer=adam, 
                      loss='binary_crossentropy', 
                      metrics=['binary_accuracy'])
        
        # Initialize callbacks
        checkpoint = ModelCheckpoint(BEST_MODEL_PATH, 
                                     monitor='val_loss', 
                                     save_best_only=True, 
                                     save_weights_only=False)
        
        lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                         factor=0.2, 
                                         patience=5)        
        
        early_stopping = EarlyStopping(monitor='val_loss', 
                                       patience=11)            
        
        callback_list = [checkpoint, 
                         lr_reduction, 
                         early_stopping]
    
    
    data_gen = ImageDataGenerator(rotation_range=20, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  horizontal_flip=True)
    
    steps_per_epoch = int(batch_limit[part] / BATCH_SIZE)
    
    model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                        steps_per_epoch=steps_per_epoch, 
                        epochs=N_EPOCHS,
                        callbacks=callback_list, 
                        validation_data=(X_valid, y_valid), 
                        workers=4)
    
    
    #free up memory
    del X_train

Part 20:
Epoch 1/100
257/257 [==============================] - 167s 651ms/step - loss: 0.2911 - binary_accuracy: 0.8870 - val_loss: 0.2941 - val_binary_accuracy: 0.8791
Epoch 2/100
257/257 [==============================] - 141s 548ms/step - loss: 0.2845 - binary_accuracy: 0.8874 - val_loss: 0.2927 - val_binary_accuracy: 0.8798
Epoch 3/100
257/257 [==============================] - 138s 539ms/step - loss: 0.2836 - binary_accuracy: 0.8905 - val_loss: 0.2929 - val_binary_accuracy: 0.8791
Epoch 4/100
257/257 [==============================] - 138s 538ms/step - loss: 0.2877 - binary_accuracy: 0.8907 - val_loss: 0.2925 - val_binary_accuracy: 0.8795
Epoch 5/100
257/257 [==============================] - 143s 557ms/step - loss: 0.2857 - binary_accuracy: 0.8912 - val_loss: 0.2928 - val_binary_accuracy: 0.8779
Epoch 6/100
257/257 [==============================] - 139s 539ms/step - loss: 0.2834 - binary_accuracy: 0.8887 - val_loss: 0.2931 - val_binary_accuracy: 0.8791
Epoch 7/100
257/257 [====

Read in the test data and evaluate the model.

In [8]:
X_test = np.load('test_set_hmgd_arr.npy')

In [9]:
model = load_model('best_model.hdf5')
model.evaluate(X_test, test_set['gender'].values)

7621/7621 [==============================] - 36s 5ms/step


[0.3071273387831475, 0.8758693084896995]

In [10]:
# free up memory
del X_test